In [1]:
import os
import sys
script_dir = os.getcwd()
module_path = os.path.abspath(os.path.join(script_dir, '..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from model import LII_LSTM, Distiller_LII_LSTM
import torch
from torch import nn
from torch.utils import data as data_utils
import distiller
from distiller.modules import DistillerLSTM as LSTM
from tqdm import tqdm # for pretty progress bar
import numpy as np

In [3]:
# Set up device and manual seed
torch.manual_seed(1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Loading the model and converting to our own implementation.

In [4]:
rnn_model = torch.load('../golden_data/20190428_checkpoint.pth.tar.best')
rnn_model = rnn_model.to(device)
rnn_model

LII_LSTM(
  (rnn): LSTM(1, 12)
  (linear): Linear(in_features=12, out_features=1, bias=True)
)

In [5]:
def get_dataset():
    index = np.arange(0, 255, 1)
    bsz = len(index)
    target = np.arange(0, 255, 1)
    index, target = torch.from_numpy(index).float().to(device), torch.from_numpy(target).float().to(device)
    print('input.size={}, target.size={}'.format(index.size(), target.size()))
    loader = data_utils.TensorDataset(index, target)
    data_loader = data_utils.DataLoader(loader, batch_size=bsz, shuffle=False)
    return data_loader

In [6]:
val_dataloader = get_dataset()
index, target = next(iter(val_dataloader))
index = index.view(-1, 1, 1)
target = target.view(-1, 1, 1)

input.size=torch.Size([255]), target.size=torch.Size([255])


In [7]:
hidden = rnn_model.init_hidden(1)
rnn_model.eval()
predictions, hidden = rnn_model(index, hidden)

In [8]:
def compute_accuracy(predictions, target):
    t1 = predictions.type(torch.LongTensor).numpy()
    t2 = target.type(torch.LongTensor).numpy()
#     print('target={}'.format(target.view(1, -1)))
#     print('predictions={}'.format(predictions.view(1, -1)))
    return np.sum(t1 == t2) / np.size(t2) * 100.

In [9]:
accuracy = compute_accuracy(predictions, target)
print(f'accuracy={accuracy:.1f}%')

accuracy=58.8%


In [10]:
def manual_model(pytorch_model_: LII_LSTM):
    rnn_type, nlayers, ninp, nhid = \
        pytorch_model_.rnn_type, \
        pytorch_model_.nlayers, \
        pytorch_model_.ninp, \
        pytorch_model_.nhid

    model = Distiller_LII_LSTM(rnn_type=rnn_type, ninp=ninp, nhid=nhid, dropout=0).to(device)
    model.eval()
    model.linear.weight = nn.Parameter(pytorch_model_.linear.weight.clone().detach())
    model.linear.bias = nn.Parameter(pytorch_model_.linear.bias.clone().detach())
    model.rnn = LSTM.from_pytorch_impl(pytorch_model_.rnn)
#    model.rrn = pytorch_model_.rnn
    return model

man_model = manual_model(rnn_model)
torch.save(man_model, '../models/manual.checkpoint.pth.tar')
man_model

Distiller_LII_LSTM(
  (rnn): DistillerLSTM(1, 12, num_layers=1, dropout=0.00, bidirectional=False)
  (linear): Linear(in_features=12, out_features=1, bias=True)
)

In [11]:
man_model = torch.load('../models/manual.checkpoint.pth.tar')

In [12]:
hidden = man_model.init_hidden(1)
man_model.eval()
predictions, hidden = man_model(index, hidden)

In [13]:
accuracy = compute_accuracy(predictions, target)
print(f'accuracy={accuracy:.1f}%')

accuracy=58.8%


In [14]:
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

criterion = torch.nn.MSELoss()
def evaluate(model, dataloader):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    hidden = model.init_hidden(1)
    with torch.no_grad():
        index, target = next(iter(dataloader))
        index = index.view(-1, 1, 1)
        target = target.view(-1, 1, 1)
        predictions, hidden = model(index, hidden)
        loss = criterion(predictions, target)
        repackage_hidden(hidden)
    return loss

In [15]:
import os
from distiller.data_loggers import QuantCalibrationStatsCollector, collector_context

man_model = torch.load('../models/manual.checkpoint.pth.tar')
distiller.utils.assign_layer_fq_names(man_model)
collector = QuantCalibrationStatsCollector(man_model)

if not os.path.isfile('manual_lstm_pretrained_stats.yaml'):
    with collector_context(collector) as collector:
        val_loss = evaluate(man_model, val_dataloader)
        collector.save('manual_lstm_pretrained_stats.yaml')

In [16]:
from distiller.quantization import PostTrainLinearQuantizer, LinearQuantMode
from copy import deepcopy

# Load and evaluate the baseline model.
man_model = torch.load('../models/manual.checkpoint.pth.tar')
val_loss = evaluate(man_model, val_dataloader)
print(f'val_loss={val_loss:5.3f}')

val_loss=4.797


In [17]:
# Define the quantizer
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats='./manual_lstm_pretrained_stats.yaml')

# Quantizer magic:
quantizer.prepare_model()
quantizer.model

Distiller_LII_LSTM(
  (rnn): DistillerLSTM(1, 12, num_layers=1, dropout=0.00, bidirectional=False)
  (linear): RangeLinearQuantParamLayerWrapper(
    mode=SYMMETRIC, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=NONE, per_channel_wts=False
    preset_activation_stats=True
    w_scale=6.2256, w_zero_point=0.0000
    in_scale=127.0000, in_zero_point=0.0000
    out_scale=0.5094, out_zero_point=0.0000
    (wrapped_module): Linear(in_features=12, out_features=1, bias=True)
  )
)

In [18]:
val_loss = evaluate(quantizer.model.to(device), val_dataloader)
print(f'val_loss={val_loss:5.3f}')

val_loss=271.940


In [19]:
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats='./manual_lstm_pretrained_stats_new.yaml',
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    per_channel_wts=True
)
quantizer.prepare_model()
quantizer.model

Distiller_LII_LSTM(
  (rnn): DistillerLSTM(1, 12, num_layers=1, dropout=0.00, bidirectional=False)
  (linear): RangeLinearQuantParamLayerWrapper(
    mode=ASYMMETRIC_SIGNED, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=NONE, per_channel_wts=True
    preset_activation_stats=True
    w_scale=PerCh, w_zero_point=PerCh
    in_scale=127.5000, in_zero_point=0.0000
    out_scale=1.0227, out_zero_point=128.0000
    (wrapped_module): Linear(in_features=12, out_features=1, bias=True)
  )
)

In [20]:
val_loss = evaluate(quantizer.model.to(device), val_dataloader)
print(f'val_loss={val_loss:5.3f}')

val_loss=310.006


In [22]:
model_fp16 = deepcopy(man_model).half()
val_loss = evaluate(model_fp16.to(device), val_dataloader)
print(f'val_loss={val_loss:5.3f}')

RuntimeError: Expected object of scalar type Half but got scalar type Float for argument #4 'mat1'

In [54]:
overrides_yaml = """
.*eltwise.*:
    fp16: true
encoder:
    fp16: true
decoder:
    fp16: true
"""
overrides = distiller.utils.yaml_ordered_load(overrides_yaml)
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats='./manual_lstm_pretrained_stats.yaml',
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    overrides=overrides,
    per_channel_wts=True
)
quantizer.prepare_model()
quantizer.model

Distiller_LII_LSTM(
  (rnn): DistillerLSTM(1, 12, num_layers=1, dropout=0.00, bidirectional=False)
  (linear): RangeLinearQuantParamLayerWrapper(
    mode=ASYMMETRIC_SIGNED, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=NONE, per_channel_wts=True
    preset_activation_stats=True
    w_scale=PerCh, w_zero_point=PerCh
    in_scale=127.5000, in_zero_point=0.0000
    out_scale=1.0227, out_zero_point=128.0000
    (wrapped_module): Linear(in_features=12, out_features=1, bias=True)
  )
)

In [55]:
val_loss = evaluate(quantizer.model.to(device), val_dataloader)
print(f'val_loss={val_loss:5.3f}')

val_loss=39.351


In [56]:
hidden = quantizer.model.init_hidden(1)
quantizer.model.eval()
predictions, hidden = quantizer.model(index, hidden)

In [57]:
accuracy = compute_accuracy(predictions, target)
print(f'accuracy={accuracy:.1f}%')

accuracy=5.5%


In [37]:
torch.save(quantizer.model, '../models/quantizer.checkpoint.pth.tar')
quantizer.model

Distiller_LII_LSTM(
  (rnn): DistillerLSTM(1, 12, num_layers=1, dropout=0.00, bidirectional=False)
  (linear): RangeLinearQuantParamLayerWrapper(
    mode=ASYMMETRIC_SIGNED, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=NONE, per_channel_wts=True
    preset_activation_stats=True
    w_scale=PerCh, w_zero_point=PerCh
    in_scale=127.5000, in_zero_point=0.0000
    out_scale=1.0227, out_zero_point=128.0000
    (wrapped_module): Linear(in_features=12, out_features=1, bias=True)
  )
)

In [53]:
percentile = 0.9
for name, param in quantizer.model.state_dict().items():
    if param.dim() < 2:
                # Skip biases
                continue
    bottomk, _ = torch.topk(param.abs().view(-1), int(percentile * param.numel()),
                                    largest=False, sorted=True)
    if (bottomk.data.size()[0]) > 0:
        threshold = bottomk.data[-1]
        print("parameter %s: q = %.2f" % (name, threshold))

parameter rnn.cells.0.fc_gate_x.w_scale: q = 4868.45
parameter rnn.cells.0.fc_gate_x.w_zero_point: q = 128.00
parameter rnn.cells.0.fc_gate_x.wrapped_module.weight: q = 128.00
parameter rnn.cells.0.fc_gate_h.w_scale: q = 309.96
parameter rnn.cells.0.fc_gate_h.w_zero_point: q = 54.00
parameter rnn.cells.0.fc_gate_h.wrapped_module.weight: q = 127.00
parameter linear.wrapped_module.weight: q = 128.00


In [33]:
quantizer = PostTrainLinearQuantizer(
    deepcopy(man_model),
    model_activation_stats='./manual_lstm_pretrained_stats_new.yaml',
    mode=LinearQuantMode.ASYMMETRIC_SIGNED,
    per_channel_wts=True,
    clip_acts="AVG"
)
quantizer.prepare_model()
quantizer.model

Distiller_LII_LSTM(
  (rnn): DistillerLSTM(1, 12, num_layers=1, dropout=0.00, bidirectional=False)
  (linear): RangeLinearQuantParamLayerWrapper(
    mode=ASYMMETRIC_SIGNED, num_bits_acts=8, num_bits_params=8, num_bits_accum=32, clip_acts=AVG, per_channel_wts=True
    preset_activation_stats=True
    w_scale=PerCh, w_zero_point=PerCh
    in_scale=129.2179, in_zero_point=1.0000
    out_scale=2.0086, out_zero_point=128.0000
    (wrapped_module): Linear(in_features=12, out_features=1, bias=True)
  )
)

In [34]:
val_loss = evaluate(quantizer.model.to(device), val_dataloader)
print(f'val_loss={val_loss:5.3f}')

val_loss=3061.427
